In [192]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import *
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

import pandas as pd



In [234]:
from sklearn.preprocessing._encoders import _BaseEncoder
import numpy as np

class CardinalEncoder(_BaseEncoder):
    """
    Encode categorical features as an integer array.
    The only difference between this CardinalEncoder and the OrdinalEncoder
    is unknown handling. OrdinalEncoder does not handle unknowns because
    categories are presumed to have an order, and the ordinal position of
    and unknown category cannot be inferred. Otherwise, these encoders function
    the same way.
    The input to this transformer should be an array-like of integers or
    strings, denoting the values taken on by categorical (discrete) features.
    The features are converted to cardinal integers. During transform, any 
    categories that are not recognized are grouped with a single integer label. 
    This results in a single column of integers (0 to n_categories) per feature.
    Parameters
    ----------
    categories : 'auto' or a list of array-like, default='auto'
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories should not mix strings and numeric
          values.
        The used categories can be found in the ``categories_`` attribute.
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : boolean, default True
        Whether or not to implement unknown handling. If False, unknown categories
        will raise an error. If True, then CardinalEncoder will fit a copy of X
        in which the final element is replaced with 'Unknown'. During transform,
        the original form of X is encoded. **If the final element in X is unique**,
        then no data loss will occur on a `fit_transform` call, but some data loss
        may occur on subsequent `transforms` if the data includes instances of 
        the unique final fitted element as well as other unknown elements.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting
        (in order of the features in X and corresponding with the output
        of ``transform``).
    """
    def __init__(self, categories='auto', dtype=np.float64, handle_unknown=True):
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown
        super().__init__()

    def fit(self, X, y=None):
        """
        Fit the CardinalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to determine the categories of each feature.
        y : None
            Ignored. This parameter exists only for compatibility with
            :class:`sklearn.pipeline.Pipeline`.
        Returns
        -------
        self
        """
        self._categories = self.categories # for compatibility with base encoder/OneHotEncoder's deprecations
        
        if self.handle_unknown:

            if hasattr(X, "iloc"):
                _X = X.copy()

                if hasattr(X, "name"):
                    _X = _X.to_frame()

                _X.iloc[-1,:] = 'Unknown'

            if isinstance(X, np.ndarray):
                _X = np.copy(X)
                if _X.ndim > 1:
                    _X[-1, :] = 'Unknown'
                else:
                    _X[-1] = 'Unknown'
        
        self._fit(_X)
        return self

    def transform(self, X):
        """
        Transform X to cardinal codes.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        if self.handle_unknown:
            cats_ = self.categories_

            if hasattr(X, 'iloc'):

                if hasattr(X, 'name'):
                    X = X.to_frame()

                for i, col in enumerate(X.columns):
                    X.loc[:,col] = X.loc[:, col].where(X.loc[:, col].isin(cats_[i]), 'Unknown')

            if isinstance(X, np.ndarray):
                if X.ndim > 1:
                    for i in range(X.shape[1]):
                        X_i = X[:,i]
                        X_i[~np.isin(X[:,i], cats_[i])] = 'Unknown'
                else:
                    X[~np.isin(X, cats_[0])] = 'Unknown'

        X_int, _ = self._transform(X)
        return X_int.astype(self.dtype, copy=False)

https://stackoverflow.com/questions/31259891/put-customized-functions-in-sklearn-pipeline

сustom elements of pipeline

In [216]:
df = pd.read_csv("https://s3.amazonaws.com/datarobot_public_datasets/DR_Demo_Fire_Ins_Loss_only.csv")

First lets check if the types for different columns. I was not sure is there a direct way to implements type inference within sklearn pipe interface, therefore I decided to create a function which sorts types into different groups.

### type inference

In [38]:
[("Percentage of Unique: ", len(df[d].unique())/len(df) * 100,
  "Mean number of symbols: ", df[d].str.len().mean(),
  "Col name: ", df[d].name)
 for d in df.columns if df[d].dtype == 'object']

[('Percentage of Unique: ',
  3.2046014790468362,
  'Mean number of symbols: ',
  2.0,
  'Col name: ',
  'Rating_Class'),
 ('Percentage of Unique: ',
  0.3286770747740345,
  'Mean number of symbols: ',
  1.0,
  'Col name: ',
  'Sub_Rating_Class'),
 ('Percentage of Unique: ',
  0.5751848808545604,
  'Mean number of symbols: ',
  1.0,
  'Col name: ',
  'Renewal_class'),
 ('Percentage of Unique: ',
  0.3286770747740345,
  'Mean number of symbols: ',
  1.0,
  'Col name: ',
  'Sub_Renewal_Class'),
 ('Percentage of Unique: ',
  0.24650780608052586,
  'Mean number of symbols: ',
  1.0,
  'Col name: ',
  'Commercial'),
 ('Percentage of Unique: ',
  0.16433853738701726,
  'Mean number of symbols: ',
  1.0,
  'Col name: ',
  'Renewal_Type'),
 ('Percentage of Unique: ',
  0.4930156121610517,
  'Mean number of symbols: ',
  37.26869350862777,
  'Col name: ',
  'ISO_cat'),
 ('Percentage of Unique: ',
  100.0,
  'Mean number of symbols: ',
  389.20049301561215,
  'Col name: ',
  'ISO_desc')]

I've decided to use heuristic if unique values of column constitute more than 50% of all occurances and mean len of the column value if > 100 characters it should be considered as text. All other field with characters, should be considered a category.

We could also consider some of the numeric with low rate of unique value to be categoric, but I will treat them as numeric for the sake of simplicity. 

In [39]:
[(len(df[d].unique())/len(df) * 100, df[d].name, df[d].dtype) for d in df.columns if df[d].dtype != 'object']

[(1.6433853738701727, 'loss', dtype('float64')),
 (0.4930156121610517, 'Exposure', dtype('float64')),
 (0.7395234182415776, 'Property_size', dtype('float64')),
 (0.5751848808545604, 'Residents', dtype('float64')),
 (1.1503697617091209, 'Norm_fire_risk', dtype('float64')),
 (85.53820870994248, 'Norm_monthly_rent', dtype('float64')),
 (14.95480690221857, 'Loan_mortgage', dtype('float64')),
 (10.106820049301561, 'No_claim_Years', dtype('float64')),
 (1.2325390304026294, 'Previous_claims', dtype('float64')),
 (8.874281018898932, 'Norm_area_m', dtype('float64')),
 (2.3007395234182417, 'Premium_remain', dtype('float64')),
 (61.38044371405095, 'Premium_renew', dtype('float64')),
 (14.626129827444537, 'crime_property_type', dtype('float64')),
 (14.95480690221857, 'crime_residents', dtype('float64')),
 (12.818405916187345, 'crime_area', dtype('float64')),
 (14.872637633525063, 'crime_arson', dtype('float64')),
 (14.872637633525063, 'crime_burglary', dtype('float64')),
 (7.8882497945768275, 'cri

In [52]:
def type_inference(data):
    categoric = []
    numberic = []
    text = []
    for d in data.columns:
        if data[d].dtype == 'object':
            if ((len(data[d].unique())/len(data)) > 0.5) and (data[d].str.len().mean() > 100):
                text.append(d)
            else:
                categoric.append(d)
        else:
            numberic.append(d)
    return numberic, categoric, text

In [74]:
target_feature = 'crime_risk'

### feature enginering and building pipeline

I haven't found obvious options for feature enginering, therefore I decided to use feature selection as as part of pipeline

In [120]:
data = df.copy()

In [121]:
target_feature = 'Norm_monthly_rent'

In [122]:
y = data.pop(target_feature)
X = data

In [123]:
numberci, categoric, text = type_inference(X)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=0, shuffle=True)

# y should be an ndarray
y_train = y_train.values
y_test = y_test.values


class StackingTransformer(BaseEstimator, TransformerMixin):
    """A transformer that wraps a RegressorMixin `est` predict function.
    
    It overloads `fit_transform` to do a stacked transform to avoid leakage.
    """
    def __init__(self, est):
        self.est = est
        
    def fit(self, X, y=None):
        if y is None:
            raise ValueError('{}.fit requires y to be not None'.format(self))
        self.est.fit(X, y)
        return self
    
    def transform(self, X):
        return self.est.predict(X)[:, np.newaxis]
    
    def fit_transform(self, X, y=None):
        if y is None:
            raise ValueError('{}.fit requires y to be not None'.format(self))
        self.models_ = []
        out = np.empty_like(y)[:, np.newaxis]
        for train, test in KFold(5, shuffle=True, random_state=0).split(X):
            self.models_.append(clone(self.est).fit(X[train], y[train]))
            out[test, 0] = self.models_[-1].predict(X[test])
        self.est = self.models_[0]
        return out


Looks that Ordinal Encoder doesn't know how to treat unknown values. <br>
I've found this [discussion](https://github.com/scikit-learn/scikit-learn/issues/13488) noting that it would just throw errors when finding unknown values. Therefore I decided to use custom Categorical Encoder from [this repo](https://github.com/jdraines/cardinal_encoder/blob/master/cardinal_encoder.py).

In [236]:

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="constant", fill_value=999999)),
    ])

categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value="missing_value")),
        ('ordinal', CardinalEncoder()),
        ('select_feature', SelectFromModel(RandomForestRegressor(), threshold=None))
    ])

numeric_pipeline_linear = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ])

categorical_pipeline_linear = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value="missing_value")),
        ('one_hot', OneHotEncoder(handle_unknown='ignore')),
        ('select_feature', SelectFromModel(ElasticNet(alpha=0.1), threshold=None))
    ])

text_pipeline = Pipeline(steps=[
    ('bow', CountVectorizer()),
    ('bow_reg', StackingTransformer(ElasticNet(alpha=0.1)))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numberci),
        ('cat', categorical_pipeline, categoric),
        ('text', text_pipeline, text[0])

    ])

linear_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline_linear, numberci),
        ('cat', categorical_pipeline_linear, categoric),
        ('text', text_pipeline, text[0])

    ])




In [205]:
from sklearn.ensemble import *

In [163]:
regressor = [
    AdaBoostRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
    ]

In [159]:
param_grid = { 
    'reg__n_estimators': [5,10, 15],
    'reg__criterion': ['mae', 'friedman_mse']
}

I am not sure if it does make a lot of sence to combine model selection step with hyperparameter optimization. However, it is interesting that I can do it this way.

In [187]:
for reg in regressor:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('reg', reg)])
    
    suitable_grid  = {d: param_grid.get(d) for d in param_grid if d.split("__")[1]
                      in [dd for dd in reg.get_params().keys()]}
    
    CV = GridSearchCV(pipe, suitable_grid, n_jobs= 1)
    
    CV.fit(X_train, y_train)  
    print(CV.best_params_)    
    print(f"model score: {CV.best_score_}")
    
    print(reg)


{'reg__n_estimators': 5}
model score: 0.5369803792107881
AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)
{'reg__criterion': 'mae', 'reg__n_estimators': 5}
model score: 0.6334707911347927
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
{'reg__criterion': 'friedman_mse', 'reg__n_estimators': 10}
model score: 0.520816901153717
GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', m

However, it would be much quicker with one model, therefore we can check some parameters while doing model selection and than do the in depth fine tuning with the selected model.

In [255]:
# %debug
est = Pipeline(steps=[('preprocessor', preprocessor),
                  ('rf', RandomForestRegressor(n_estimators=5, criterion='mae', n_jobs=-1))])

est.fit(X_train, y_train)

print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

model score: 769.254


In [256]:
# %debug
est = Pipeline(steps=[('preprocessor', preprocessor),
                  ('rf', GradientBoostingRegressor(n_estimators=5))])

est.fit(X_train, y_train)

print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

model score: 913.507


In [257]:
# %debug
est = Pipeline(steps=[('preprocessor', linear_preprocessor),
                  ('rf', LinearRegression())])

est.fit(X_train, y_train)

print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

model score: 761.420


At the end, we had similar results for RF and Linear Regression on this task working on test data.